In [129]:
import rdflib
import pandas as pd
import rdflib.graph as g
import rdflib.plugins.sparql as sparql
from tabulate import tabulate
import os
from SPARQLWrapper import SPARQLWrapper, JSON
from rdflib import plugin
from rdflib import Namespace, BNode, Literal, URIRef
from rdflib.namespace import *
from rdflib.graph import Graph, ConjunctiveGraph
from rdflib.plugins.memory import IOMemory
from pprint import pformat
from IPython.lib.pretty import pprint
from pandas import *
from pylab import *

In [130]:
#Getting RDF datasets 
#"Infant Mortality Rate by Race" and the "US infant Population by state/region"
g = rdflib.Graph()

#Parsing Datasets
g.parse("datasets/Infant_Mortality_byRace_Totals1.rdf")
g.parse("datasets/us_infant_pop_totals_2010_2014.rdf")
g.serialize(format='pretty-xml')
print("graph has %s statements." % len(g))

graph has 6255 statements.


In [131]:
for s, p, o in g:
   if (s, p, o) not in g:
       raise Exception("It better be!")

In [135]:
#Printing Raw RDF DCAT Data
print( g.serialize(format='pretty-xml') )

<?xml version="1.0" encoding="utf-8"?>
<rdf:RDF
  xmlns:socrata="http://www.socrata.com/rdf/terms#"
  xmlns:rdfs="http://www.w3.org/2000/01/rdf-schema#"
  xmlns:usps="http://www.w3.org/2000/10/swap/pim/usps#"
  xmlns:ds1="http://opendata.socrata.com/resource/tpsv-v8m8/"
  xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#"
  xmlns:dsbase="http://opendata.socrata.com/resource/"
  xmlns:geo="http://www.w3.org/2003/01/geo/wgs84_pos#"
  xmlns:ds="http://opendata.socrata.com/resource/rwi9-nfpf/"
>
  <dsbase:tpsv-v8m8 rdf:about="http://opendata.socrata.com/resource/tpsv-v8m8/100707">
    <ds1:popestimate2011>238</ds1:popestimate2011>
    <ds1:sex>Total</ds1:sex>
    <rdfs:member rdf:resource="http://opendata.socrata.com/resource/tpsv-v8m8"/>
    <socrata:rowID>100707</socrata:rowID>
    <ds1:region>West</ds1:region>
    <ds1:popestimate2013>267</ds1:popestimate2013>
    <ds1:origin>Total</ds1:origin>
    <ds1:popestimate2012>230</ds1:popestimate2012>
    <ds1:race>Black</ds1:race>
    <d

In [139]:
#Displaying Classes and Distinct URI's for each subject
print("\n\n------------------------ Printing Out RDF Data -----------------------------------------------\n")

qClass = g.query("""SELECT DISTINCT ?class WHERE { ?s  a ?class.} """)
#Printing Years
print ("CLASS\n------")
for r in qClass:
    print (r[0])  
print ("----------------------------------------------")
    
qProperties = g.query("""SELECT DISTINCT ?property WHERE 
{ ?s  ?property ?o.} LIMIT 25 OFFSET 0 """)
#Printing Properties
print ("URIs\n------")
for r in qProperties:
    print (r[0])
    
    



------------------------ Printing Out RDF Data -----------------------------------------------

CLASS
------
http://opendata.socrata.com/resource/tpsv-v8m8
http://opendata.socrata.com/resource/rwi9-nfpf
http://www.w3.org/2003/01/geo/wgs84_pos#SpatialThing
----------------------------------------------
URIs
------
http://opendata.socrata.com/resource/tpsv-v8m8/sex
http://opendata.socrata.com/resource/rwi9-nfpf/age_at_death
http://opendata.socrata.com/resource/tpsv-v8m8/popestimate2012
http://www.w3.org/2000/10/swap/pim/usps#zipCode
http://www.w3.org/1999/02/22-rdf-syntax-ns#type
http://opendata.socrata.com/resource/tpsv-v8m8/race
http://www.w3.org/2000/10/swap/pim/usps#deliveryAddress
http://opendata.socrata.com/resource/rwi9-nfpf/total_livebirths_denominator
http://www.w3.org/2000/01/rdf-schema#member
http://opendata.socrata.com/resource/tpsv-v8m8/census2010pop
http://opendata.socrata.com/resource/tpsv-v8m8/popestimate2013
http://www.w3.org/2003/01/geo/wgs84_pos#lat
http://www.w3.org

In [167]:
#Running Query to show the infant Mortality Rate by race in 2013
qMortality = g.query("""
    SELECT DISTINCT ?year ?race_ethnicity ?count 
    WHERE
     
        { ?s ds:year ?year. 
            ?s ds:race_ethnicity ?race_ethnicity.
            ?s ds:count ?count.
        }   

    ORDER BY DESC(?year)
    """)

#Print out Query
print("-------------------------------------------Total Deaths----------------------------------\n")
headers = ['YEAR', 'ETHNICITY','Total Deaths']
print tabulate(qMortality,headers)

-------------------------------------------Total Deaths----------------------------------

  YEAR  ETHNICITY                    Total Deaths
------  -------------------------  --------------
  2013  Asian                                 170
  2013  White/Other                           533
  2013  Pacific Islander/Hawaiian               6
  2013  Hispanic                             1212
  2013  American Indian                         9
  2013  Not Stated or Unknown                  10
  2013  Two or more races                     133
  2013  Total                                2348
  2013  Black                                 275
  2012  Total                                2247
  2012  Asian                                 198
  2012  Two or more races                      84
  2012  Hispanic                             1159
  2012  Black                                 261
  2012  American Indian                         7
  2012  Not Stated or Unknown                   7
  2012  W

In [170]:
#Running Query to show the US infant Population sorted by Region
qTest = g.query("""
    SELECT DISTINCT ?region ?state ?race ?popestimate2013 
    WHERE 
        {?s ds1:region ?region.
         ?s ds1:state ?state. 
         ?s ds1:race ?race.
         ?s ds1:popestimate2013 ?popestimate2013. FILTER(?year=\"2013\")
         }
    ORDER BY ASC (?region)
    """)
print("----------------------------Infant US Population by State----------------------------------\n")
headers = ['REGION', 'RACE', 'POPULATION','AGE']
print tabulate(qPopulation,headers)

----------------------------Infant US Population by State----------------------------------

REGION     RACE               POPULATION    AGE
---------  ---------------  ------------  -----
Midwest    Black                   13371      0
Midwest    American Indian          2242      0
Midwest    Black                    8289      0
Midwest    Asian                    1724      0
Midwest    Asian                    4907      0
Midwest    Native Hawaiian            30      0
Midwest    Asian                    1405      0
Midwest    Black                   13081      0
Midwest    Native Hawaiian           133      0
Midwest    Black                    3124      0
Midwest    Asian                     281      0
Midwest    American Indian          1164      0
Midwest    White                  111485      0
Midwest    White                   57573      0
Midwest    White                   35388      0
Midwest    Native Hawaiian            89      0
Midwest    White                   10065   

In [169]:
#Running Query to show the infant Mortality Rate by race from 2004 - 2013
qMortality = g.query("""
    SELECT DISTINCT ?year ?race_ethnicity ?count ?total_livebirths_denominator ?rate 
    WHERE
    { 
        { ?s ds:year ?year. 
            ?s ds:race_ethnicity ?race_ethnicity.
            ?s ds:count ?count.
            ?s ds:total_livebirths_denominator ?total_livebirths_denominator.
            ?s ds:rate ?rate. 
        }   
        UNION
        {
            ?popestimate2013 ds1:popestimate2013 ?popestimate2013 .
            ?region ds1:region ?region.
        } 
    }ORDER BY DESC(?year)
    """)

#Print out Query
print("----------------------------Infant Mortality Rate----------------------------------")
headers = ['YEAR', 'ETHNICITY','Total Deaths','Total Births','Total Deaths Rate']
print tabulate(qMortality,headers)

----------------------------Infant Mortality Rate----------------------------------
  YEAR  ETHNICITY                    Total Deaths    Total Births    Total Deaths Rate
------  -------------------------  --------------  --------------  -------------------
  2013  Asian                                 170           68117                  2.5
  2013  White/Other                           533          137896                  3.9
  2013  Pacific Islander/Hawaiian               6            1994                  3
  2013  Hispanic                             1212          238200                  5.1
  2013  American Indian                         9            1794                  5
  2013  Not Stated or Unknown                  10            9385                  1.1
  2013  Two or more races                     133           11169                 11.9
  2013  Total                                2348          494392                  4.7
  2013  Black                                 275 

In [165]:
#Combining Datasets to find which race has the highest Mortality rate for 2013
qMortality = g.query("""
    SELECT DISTINCT ?year ?race_ethnicity ?count ?total_livebirths_denominator ?rate 
    WHERE { 
       {?s ds:year ?year. ?s ds:race_ethnicity ?race_ethnicity.?s ds:count ?count.
        ?s ds:total_livebirths_denominator ?total_livebirths_denominator.
        ?s ds:rate ?rate. FILTER(?year = \"2013\")}   
        UNION
        {?popestimate2013 ds1:popestimate2013 ?popestimate2013 .?region ds1:region ?region. } }ORDER BY DESC(?year)""")

#Print out Query
print("----------------------------Infant Mortality Rate 2013----------------------------------\n")
headers = ['YEAR', 'ETHNICITY','Total Deaths','Total Births','Total Deaths Rate']
print tabulate(qMortality,headers)

----------------------------Infant Mortality Rate 2013----------------------------------

  YEAR  ETHNICITY                    Total Deaths    Total Births    Total Deaths Rate
------  -------------------------  --------------  --------------  -------------------
  2013  Asian                                 170           68117                  2.5
  2013  White/Other                           533          137896                  3.9
  2013  Pacific Islander/Hawaiian               6            1994                  3
  2013  Hispanic                             1212          238200                  5.1
  2013  American Indian                         9            1794                  5
  2013  Not Stated or Unknown                  10            9385                  1.1
  2013  Two or more races                     133           11169                 11.9
  2013  Total                                2348          494392                  4.7
  2013  Black                               